# Notebook 3: A first look into search
In this notebook we will take a look at Luminous Explore.

Luminous Explore is our model for semantic similarity.

With Luminous Explore you can use the meaning of text to create awesome applications.

Try it out below!

In [ ]:
!pip install aleph_alpha_client

# Some additional imports for search with Luminous Explore
from typing import Sequence
from aleph_alpha_client import ImagePrompt, AlephAlphaClient, AlephAlphaModel, SemanticEmbeddingRequest, SemanticRepresentation, Prompt
from IPython.display import Image  
import math
import os

## The model for embedding

Our Embedding model is called Luminous Explore.

However to access its functionalities, it is important to define luminous-base as the used model.

In [ ]:
# instantiate the client and model
model = AlephAlphaModel.from_model_name(model_name="luminous-base", token="API-TOKEN")

### Simple functions for embedding and searching
Here we provide a simple function for embedding text and a simple function for calculating the similarity between two texts.

Please take a moment to understand what each function does.

The cosine similarity is a measure of similarity between two vectors of an inner product space that measures the cosine of the angle between them.

You don't have to understand the details of the code, but you should understand the general idea.

In [ ]:
# function for symmetric embedding
def embed_symmetric(text: str):
    # Create an embeddingrequest with the type set to symmetric
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric)
    # create the embedding
    result = model.semantic_embed(request)
    return result.embedding

# function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

### Using semantic similarity
Now we can use the semantic similarity to find similar texts.

Let's compare two texts, one in English and one in Italian to see how it works.

In this case, both sentences have the same meaning. So they should return a high similarity score.

In [ ]:
# define the texts
text_a = "The sun is shining"
text_b = "Il sole splende"

# show the similarity
print(cosine_similarity(embed_symmetric(text_a), embed_symmetric(text_b)))

## The embedding
Let's also take a look at the embedding itself and how it looks.

In the cell below we print the first 100 elements of the embedding.

The embedding is 5120 elements long, so printing all of it would be quite a lot.

In [ ]:
%%time
print(embed_symmetric(text_a)[:100])
print("\n")
print(embed_symmetric(text_b)[:100])

## Multimodal Similarity
Amazingly, as luminous supports multimodal input, we can even semantically compare texts to images with Luminous Explore.

This is not an explicitly developed feature but rather an emergent property of the model.

*Please keep in mind, that multi-modal semantic similarity is probably less robust than text to text similarity.*

In [ ]:
# get an image from a url
url = "https://cdn-images-1.medium.com/max/1200/1*HunNdlTmoPj8EKpl-jqvBA.png"

# put the image into a prompt
image_prompt = ImagePrompt.from_url(url)

# display the image from the url
Image(url=url, width=300, height=300)

In [ ]:
# create a positive and negative text example
positive_example = "A neural network Architecture with Attention and Embeddings"
negative_example = "An image of a beatuiful beach"

# Calculate the embedding for the positive and negative example
print(f"The score for the positive example is: {cosine_similarity(embed_symmetric(positive_example), embed_symmetric(image_prompt))}")
print(f"The score for the neagtive example is: {cosine_similarity(embed_symmetric(negative_example), embed_symmetric(image_prompt))}")